In [ ]:
# pip install opencv-python==4.9.0.80 
# pip install numpy==1.26.4
# pip install matplotlib==3.9.0
# pip install networkx==3.3
# pip install scipy==1.13.0

import os
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
import networkx as nx
from scipy.sparse.csgraph import connected_components

# Função que recebe uma imagem colorida como entrada e retorna a mesma imagem convertida para escala de cinza
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Converter a imagem de um espaço de cores RGB para escala de cinza
    return gray

# Função que usará um modelo para detectar o rosto de uma pessoa
def detect_faces(image, face_classifier):
    faces = face_classifier.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 50))
    return faces

# Função que usará um modelo para detectar os olhos de uma pessoa
def detect_eyes(image, eyes_classifier):
    eyes = eyes_classifier.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 50))
    return eyes

# Função que usará um modelo para detectar o nariz de uma pessoa
def detect_noses(image, nose_classifier):
    noses = nose_classifier.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 50))
    return noses

# Função que usará um modelo para detectar a boca de uma pessoa
def detect_mouths(image, mouth_classifier):
    mouths = mouth_classifier.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 50))
    return mouths

# Função que irá nos retornar os contornos de uma imagem
def apply_canny(image):
    edges = cv2.Canny(image, 150, 200)
    return edges

# Função que plotará imagens
def plots(figura, title=''):
    plt.figure(figsize=(20,10))
    plt.imshow(figura, cmap='gray')
    plt.axis('off')
    plt.title(title)
    plt.show()

# Função que aleatoriza 
def retirando_pixels(mEdges, thresh):
    lCoord = [] # Lista para armazenar as coordenddas dos pixels de interesse

    for i in range(mEdges.shape[0]):
        for j in range(mEdges.shape[1]):
            if mEdges[i][j] == 255: # Verificando se o ponto encontrado corresponde ao pixel 255
                lCoord.append((i,j)) # Adicionando a coordenada desse pixel encontrado no lCoord
    mainCoord = lCoord.copy() # Copiando a lista de coordenadas dos pontos (coordenadas dos pixels de interesse)
    random.shuffle(lCoord) # Aleatorizando a ordem das coordenadas dos pontos

    times = thresh # Definando um limiar para zerarmos os pixels

    #for i,t in zip(lCoord, range(times)): # Zipamos a lista de coordenadas com o limiar para definirmos um 'critério de parada' no laço
    #        mEdges[i[0]][i[1]] = 0 # Zerando os pixels 255

    return mEdges, mainCoord

# Função que fará o processamento de imagem e nos retornará os contornos das características
def process_image(image_path, face_classifier, eyes_classifier, nose_classifier, mouth_classifier):
    image = cv2.imread(image_path) # Lendo a imagem
    processed_image = preprocess_image(image) # Pré-processando a imagem (convertendo para escala de cinza)
    faces = detect_faces(processed_image, face_classifier) # Detectando o rosto na imagem

    for (x, y, w, h) in faces:
        face_roi = processed_image[y:y+h, x:x+w] # Recortando a região de interesse (Roi) do rosto
        noses = detect_noses(face_roi, nose_classifier) # Detectando o nariz na região de interesse do rosto

        for (nx, ny, nw, nh) in noses:
            nose_roi = face_roi[ny:ny+nh, nx:nx+nw] # Recortando a região de interesse (Roi) do nariz
            plots(nose_roi, title="ROI Nariz") # Plotando a região de interesse do nariz
            nariz_contornos = apply_canny(nose_roi) # Aplicando o detector de bordas de Canny na região de interesse do nariz
            plots(nariz_contornos, title="Contornos com Canny")
            newNoseEdges, mainCoord = retirando_pixels(nariz_contornos, 100)

    return nariz_contornos, mainCoord

# Função para processar a matriz e retornar as ligações entre os pontos
def processar_matriz(matriz, limViz1, limViz2):
    def coord_pts(matriz):
        dAuxiliar = {}  # Um dicionário auxiliar para armazenar as coordenadas dos pontos
        pto = 0  # Um contador para contabilizarmos os pontos
        for i in range(matriz.shape[0]):
            for j in range(matriz.shape[1]):
                if matriz[i][j] == 255:  # Verificando se o ponto encontrado corresponde ao pixel 255
                    dAuxiliar[(i, j)] = pto  # Adicionando esse pixel encontrado no dAuxiliar
                    pto += 1
        return dAuxiliar

    def linha_coluna(ponto1, ponto2):
        x1, y1 = ponto1
        x2, y2 = ponto2
        return x1 == x2 or y1 == y2

    def diagonal(ponto1, ponto2):
        x1, y1 = ponto1
        x2, y2 = ponto2
        return abs(x1 - x2) == abs(y1 - y2)

    def distancia_absoluta(ponto1, ponto2):
        distancia_x = abs(ponto1[0] - ponto2[0])
        distancia_y = abs(ponto1[1] - ponto2[1])
        return distancia_x + distancia_y

    def vizinhanca(ponto1, ponto2, limite):
        distancia = distancia_absoluta(ponto1, ponto2)
        return distancia <= limite

    def verificar_vizinhanca(pontos, limViz1, limViz2):
        ligacoes = {}  # Dicionário para armazenar as ligações entre os pontos
        for ponto1, id1 in pontos.items():
            ligacoes[id1] = []  # Inicializa a lista de ligações para o ponto atual
            for ponto2, id2 in pontos.items():
                if ponto1 != ponto2:  # Evitar calcular a distância do ponto com ele mesmo
                    if linha_coluna(ponto1, ponto2):  # Verificando se os pontos estão na mesma linha/coluna
                        if vizinhanca(ponto1, ponto2, limViz1):  # Verificando se as distâncias estão dentro do limite imposto
                            ligacoes[id1].append(id2)  # Adicionar a ligação entre os pontos ao dicionário de ligações

        for ponto1, id1 in pontos.items():
            for ponto2, id2 in pontos.items():
                if ponto1 != ponto2:  # Evitar calcular a distância do ponto com ele mesmo
                    if diagonal(ponto1, ponto2):  # Evitar calcular a distância do ponto com ele mesmo
                        if vizinhanca(ponto1, ponto2, limViz2):  # Verificando se as distâncias estão dentro do limite imposto
                            ligacoes[id1].append(id2)  # Adicionar a ligação entre os pontos ao dicionário de ligações
        return ligacoes

    pontos = coord_pts(matriz)
    resultado = verificar_vizinhanca(pontos, limViz1, limViz2)
    return resultado

# Função para criar a matriz de adjacências
def matriz_adjacencias(graph_dict):
    n = max(graph_dict.keys()) + 1 # Encontra o maior número de nó no grafo e adiciona 1 a ele para ajustar a indexação que começa em 0 no python
    M = np.zeros((n,n)) # Uma matriz de adjacência quadrada n = qtde pixels 255
    print(M)
    
    for node, neighbors in graph_dict.items(): #Itera sobre o dicionário onde cada chave representa um nó e o valor é uma lista de vizinhos desse nó.
        for neighbor in neighbors: # Itera sobre a lista de vizinhos do nó atual.
            M[node][neighbor] = 1 # Define a entrada correspondente na matriz de adjacência como 1 para indicar uma aresta entre o nó node e o vizinho neighbor.
    
    return M

def main():
    directory = './image'
    face_classifier = cv2.CascadeClassifier("./models/haarcascade_frontalface_default.xml")
    eyes_classifier = cv2.CascadeClassifier("./models/haarcascade_eye.xml")
    nose_classifier = cv2.CascadeClassifier("./models/haarcascade_mcs_nose.xml")
    mouth_classifier = cv2.CascadeClassifier("./models/haarcascade_mcs_mouth.xml")
    
    all_mAdjNose = []

    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".ppm"):
            image_path = os.path.join(directory, filename)
            mNoseEdges, mainCoord = process_image(image_path, face_classifier, eyes_classifier, nose_classifier, mouth_classifier)
            dLigacoes = processar_matriz(mNoseEdges, limViz1 = 1, limViz2 = 2)
            mAdjNose = matriz_adjacencias(dLigacoes)
            print(len(mAdjNose[28:]))

            for i in range(346):
                print(i, mAdjNose[31][i])

            all_mAdjNose.append(mAdjNose)
    return all_mAdjNose, mainCoord, image_path

if __name__ == "__main__": 
    all_mAdjNose, mainCoord, image_path = main()


In [ ]:
# Função para encontrar as principais componentes conectados
def main_connected_components(all_mAdjNose):

    ## Encontrar os componentes conectados na matriz de adjacências
    comp_conectadas, labels = connected_components(all_mAdjNose[0]) # Encontrar os componentes conectados na matriz de adjacências  (comp_conectadas = número de componentes conectados, labels = rótulos dos componentes conectados)

    ## Criando um dicionário para armazenar os rótulos das componentes conectadas e seus respectivos índices
    connectedComponentLabels = {} 

    for i in range(len(labels)): # Iterar sobre os labels e seus respectivos índices
        if labels[i] not in connectedComponentLabels: # Verificar se o label atual já está no dicionário
            connectedComponentLabels[labels[i]] = [i] # Se não estiver, adiciona o índice atual em uma lista
        else:
            connectedComponentLabels[labels[i]].append(i) # Se já estiver, adiciona o índice atual à lista de índices

    print('Dicionário de labels:', connectedComponentLabels)

    ## Dicionário auxiliar para armazenar o número de nó em cada conexo
    componentNodeCount = {}

    for i in connectedComponentLabels.keys(): # Iterar sobre as chaves do dicionário de labels
        componentNodeCount[i] = len(connectedComponentLabels[i]) # Adicionar o número de conectividades em cada conexo ao dicionário auxiliar
    print('Dicionário com a quantidade de nós:',componentNodeCount) 

    ## Encontrar o maior e o menor número de nós em um conexo
    maxNodeComponent = max(componentNodeCount, key=componentNodeCount.get)
    print('Maior nº de nós em um conexo:', componentNodeCount[maxNodeComponent])
    minNodeComponent = min(componentNodeCount, key=componentNodeCount.get) 
    print('Menor nº de nós em um conexo:', componentNodeCount[minNodeComponent])

    # Lista para armazenar as chaves (componentes conexas) que serão deletadas
    componentsToRemove = []

    for key in componentNodeCount.keys(): # Iterar sobre as chaves do dicionário que armazena o número de nós em cada componente
        if componentNodeCount[key] < round(0.2 * componentNodeCount[maxNodeComponent]): # Verificar se o número de nós é menor que 20% do maior número de nós
            componentsToRemove.append(key) # Adicionar a componente à lista de componentes que serão deletadas

    ## Filtrar as componentes conectadas mantendo apenas as principais
    for key in componentsToRemove: # Iterar sobre a lista de componentes que serão deletadas
        del componentNodeCount[key] # Deletar a componente do dicionário componentNodeCount

    listMainComponents = componentNodeCount.keys() # Lista para armazenar as componentes que serão mantidas
    print('Lista das chaves que serão filtradas:', listMainComponents)

    ## Dicionário para armazenar os principais componentes conectados e seus respectivos nós
    dicMainComponents = {}

    for key in listMainComponents: # Iterar sobre as chaves (componentes) que serão mantidas
        dicMainComponents[key] = connectedComponentLabels[key] # Adicionar os nós dos componentes que serão mantidos ao dicionário dos principais componentes conectados

    print('Dicionário com os labels filtrados:', dicMainComponents)
    return dicMainComponents

In [96]:
def highlight_components(mainCoord, dicMainComponents):
    '''
    Função para plotar os pontos das componentes conectadas

    - mainCoord: lista de coordenadas dos pontos de interesse
    - dicMainComponents: dicionário com os índices dos pontos filtrados por componente conectada
    '''
    colors = ['blue', 'red', 'green', 'yellow', 'black', 'orange']
    plt.figure(figsize=(8, 6))
    
    for color,label in enumerate(dicMainComponents.keys()): # Iterar sobre as chaves do dicionário que contém os principais componentes conectados
        for idx in dicMainComponents[label]: # Iterar sobre os nós dos componentes conectados
            i, j = mainCoord[idx] # Obter as coordenadas dos pontos
            # print(i, j, idx)
            plt.scatter(j, i, color=colors[color%len(colors)])  # Plotar os pontos filtrados
    
    plt.title("Pontos das Componentes Conectadas")
    plt.xlabel("Coluna")
    plt.ylabel("Linha")
    plt.gca().invert_yaxis()  # Inverter o eixo y para corresponder à orientação das imagens OpenCV
    plt.grid(True)
    plt.show()

In [ ]:
dicMainComponents = main_connected_components(all_mAdjNose)
highlight_components(mainCoord, dicMainComponents)

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

nodes = dicLabels[3]

componente_3 = all_mAdjNose[0][np.ix_(nodes,nodes)]
minimum_spanning_tree(componente_3).toarray().astype(int)